In [1]:
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt
import neuro_morpho_toolbox as nmt

D:\ProgramData\Anaconda3\lib\site-packages\neuro_morpho_toolbox-0.1-py3.7.egg\neuro_morpho_toolbox\
Loading CCF Atlas data...
Loading time: 0.45
Loading CCF brain structure data...
Loading time: 0.00
Loading selected CCF Atlas and Contour data...
Loading time: 0.84


In [2]:
mouse = pd.read_csv("Mouse.csv", usecols=[1,2,4,5,6], index_col=[0])
mouse.head()

,Acronym,red,green,blue
ID,,,,
997,root,255,255,255
8,grey,191,218,227
567,CH,176,240,255
688,CTX,176,255,184
695,CTXpl,112,255,112


In [3]:
img1 = sitk.ReadImage("./annotation_25.nrrd") # load annotation brain
print(img1.GetDimension())
print(img1.GetSize())
print(img1.GetSpacing())

array1 = sitk.GetArrayFromImage(img1)

3
(528, 320, 456)
(25.0, 25.0, 25.0)


In [4]:
value, counts = np.unique(array1, return_counts=True) # value: get annotated brain id in annotation brain
uid = [i for i in value if i in mouse.index.tolist()] # uid：select brain id that exist in both annotation brain and mouse.csv
# print(len(value), len(uid))

In [18]:
# go to Mouse.csv and select re-color region's id 

# eg: 
# CTX: 688
# LSX: 275
# TH: 549
# HPF: 1089
# HY: 1097
# CBX: 528
# CP: 672
# MB: 313

# put the selected ID into a list
areas_ids=[512,688,343,56,275,1089,528,672, 275]

childs={}# to save each brain area's child brain region id 
for area in areas_ids:
    childs[area]=(nmt.bs.get_all_child_id(area))
    for area_ in areas_ids:
        if area_==area:
            continue
        else:
            if area_ in childs[area]: # 
                print("brain region ",area_,"  in brain region ",area)
array2 = np.zeros(list(array1.shape)+[3], dtype=np.int16) # init a brain array2 with zero

brain region  528   in brain region  512
brain region  1089   in brain region  688


In [19]:
print(childs.keys()) # make sure that big brain region is recolor first and its child brain region get recolored later

dict_keys([512, 688, 343, 56, 275, 1089, 528, 672])


In [20]:
child_uids=[] # final brain region id list (to be recolor)
for k,v in childs.items():
    print(k)
    child_uids.append([i for i in uid if (i in v)])

512
688
343
56
275
1089
528
672


In [21]:
for child_uid in child_uids:
    print(mouse.loc[int(child_uid[0]), "red"], mouse.loc[int(child_uid[0]), "green"], mouse.loc[int(child_uid[0]), "blue"])
    for cur_id in child_uid:
        idx = np.where(array1==cur_id)
        array2[idx[0], idx[1], idx[2], 0] = mouse.loc[int(child_uid[0]), "red"]
        array2[idx[0], idx[1], idx[2], 1] = mouse.loc[int(child_uid[0]), "green"]
        array2[idx[0], idx[1], idx[2], 2] = mouse.loc[int(child_uid[0]), "blue"]

# 

img2 = sitk.GetImageFromArray(array2)
sitk.WriteImage(img2, "annotation_recolor.nrrd")

255 253 188
24 128 100
255 76 62
128 205 248
144 203 237
126 208 75
255 252 145
152 214 249
